<style>
h1 {text-align: center;}
</style>
<h1> Accelerating Supply Chain Using Ant Colony Optimization </h1>

# 1. Introduction

The problem at hand is optimizing inventory management and shipping decisions for a network of 50 stores, taking into account factors such as demand, current inventory levels, shipping costs, and replenishment costs. To tackle this problem, we will be using Ant Colony Optimization (ACO) - a metaheuristic algorithm inspired by the foraging behavior of ants. ACO is a powerful optimization technique that has been successfully applied to a variety of problems in diverse fields such as transportation, logistics, and telecommunications. By leveraging ACO, we aim to find the best route and associated costs for shipping goods between stores while minimizing the overall replenishment costs.


# 2. Code Breakdown



The code can be broken down into three main parts: setting up the problem-specific parameters, setting up the ACO algorithm parameters, and running the ACO algorithm using parallel and independent ants.



## 2.1. Setting up the problem-specific parameters



This part of the code defines the parameters that are specific to the multi-store inventory management problem. These include the number of stores in the system (num_stores), the demand at each store `demand`, the current inventory at each store `inventory`, the lead time for each store `lead_time` the shipping cost matrix `shipping_cost` and the cost of replenishing inventory at each store `replenishment_cost`.
Lead time refers to the amount of time it takes for a shipment to arrive at a store after an order has been placed. Specifically, the `lead_time` variable in the code represents the lead time for each store in the network, and is randomly generated for each store. The lead time is an important factor to consider in inventory management, as it affects the timing of when orders need to be placed in order to ensure that inventory levels remain sufficient.

The `np.random.randint()` function is used to generate random integers between specified minimum and maximum values for the `demand`, `inventory` and `lead_time` arrays. The `shipping_cost` matrix is generated using the `np.random.rand()` function to create a matrix of random floats between 0 and 1, representing the cost of shipping from one store to another. Finally, the `replenishment_cost` array is also generated using the `np.random.rand()` function.

In [58]:
# Imports
import numpy as np
import pandas as pd
from IPython.display import display
from joblib import Parallel, delayed ,parallel_backend # import joblib for parallel processing
import time

np.random.seed(142)

# Define problem-specific parameters
num_stores = 50
demand = np.random.randint(1, 100, num_stores)  # demand at each store
inventory = np.random.randint(1, 1000, num_stores)  # current inventory at each store
lead_time = np.random.randint(1, 10, num_stores)  # lead time for each store
shipping_cost = np.random.rand(num_stores, num_stores)  # shipping cost matrix
replenishment_cost = np.random.rand(num_stores)  # cost of replenishing inventory at each store

## 2.2. Setting up the ACO algorithm parameters

This part of the code defines the parameters that control the behavior of the ant colony optimization `ACO` algorithm. These include the number of ants in the colony `num_ants`, the relative importance of pheromone trail and distance `alpha` and `beta`, respectively, the pheromone evaporation rate `rho`, the amount of pheromone deposited by ants `q`, and the number of iterations to run the algorithm `num_iterations`.

The pheromone trail is initialized to a matrix of ones with dimensions (`num_stores`, `num_stores`). This represents the initial level of pheromone on all possible routes between stores.

In [59]:
# Define ACO parameters
num_ants = 20
alpha = 1  # relative importance of pheromone trail
beta = 2   # relative importance of distance . (It is a common choice in ACO implementations and has been found to be effective in many applications.)
rho = 0.1  # pheromone evaporation rate
q = 1      # amount of pheromone deposited by ants
num_iterations = 100

# Initialize pheromone trail
pheromone = np.ones((num_stores, num_stores))

## 2.3. Defining the probability of choosing a store in the next step


The `prob` function calculates the probability of choosing a particular store given the current store and the set of visited stores, based on the pheromone trail and the distance between stores. The formula for calculating the probability of choosing a particular store is as follows:
$$
P_{ij} = \frac{(\tau_{ij})^\alpha . (\eta_{ij})^\beta}{\sum_{l \in J_i^u }(\tau_{il})^\alpha . (\eta_{il})^\beta}
$$
Where:

- $P_{ij}$: the probability of moving from store i to store j

- $\tau_{ij}$: the amount of pheromone on the path between stores i and j

- $\alpha$: a parameter that controls the relative importance of the pheromone values

- $\eta_{ij}$: a value that represents the attractiveness of the path between stores i and j, defined as:
 $\eta_{ij}$=$(\frac{1}{(SHIPPING-COST_{ij} + REPLENISHMENT-COST_{j})})$

- $\beta$: a parameter that controls the relative importance of the distance and replenishment cost

- $J_i^u$: the set of all unvisited stores from store i

In [60]:
# Define function to calculate the probability of choosing a particular store
def prob(store, visited_stores, pheromone, alpha, beta):
    unvisited_stores = np.delete(np.arange(num_stores), visited_stores)
    numerator = np.power(pheromone[store, unvisited_stores], alpha) * np.power(1/(shipping_cost[store, unvisited_stores] + replenishment_cost[unvisited_stores]), beta)
    denominator = np.sum(numerator)
    return numerator / denominator

## 2.4. Updating the pheromone trails on the ant routes


The `update_pheromone` function updates the pheromone trail based on the routes taken by the ants and their associated costs. The update formula for the pheromone level between 2 stores is:

$PHEROMONE_{ij}$ = $(1-\rho).PHEROMONE_{ij} + \frac{q}{ANT-COST_i}$

Where:

- $PHEROMONE_{ij}$ : the current pheromone level on the edge between `i` & `j`.

- $\rho$ : the pheromone evaporation rate.

- q : a constant that represents the strength of the pheromone deposit.

- $ANT-COST_i$ : the cost of the route taken by ant i.

Note : Dividing `q` by $ANT-COST_i$ in the `update_pheromone` function is a way to scale the pheromone update based on the cost of the ant's route. The `q` parameter represents the amount of pheromone deposited by the ant on its route, and dividing it by the cost of the route ensures that more pheromone is deposited on shorter routes, and less pheromone is deposited on longer routes. This helps to reinforce the shortest routes and discourage ants from exploring longer, less efficient routes.

In [61]:
# Define function to update the pheromone trail
def update_pheromone(pheromone, ant_routes, ant_costs, rho, q):
    for i in range(num_ants):
        for j in range(num_stores-1):
            store1, store2 = ant_routes[i,j], ant_routes[i,j+1]
            pheromone[store1, store2] = (1-rho) * pheromone[store1, store2] + q/ant_costs[i]
        last_store, first_store = ant_routes[i,-1], ant_routes[i,0]
        pheromone[last_store, first_store] = (1-rho) * pheromone[last_store, first_store] + q/ant_costs[i]

## 2.5. Running the Independent ACO algorithm



This part of the code runs the main `run_aco_independent` function, which implements the ant colony optimization algorithm. The function first initializes the `best_route` and `best_cost` variables to None and infinity, respectively.

In the for loop, the algorithm generates a set of random routes for each ant in the colony, records their associated costs, updates the pheromone trail and finds the best route found by the ants.

The inner loop of the for loop uses the prob function to calculate the probability of choosing each unvisited store given the current store, and then selects the next store based on these probabilities. The code also calculates the shipping time, order quantity, shipping cost to the next store and the replenishment cost for the next store.

If the cost of the route taken by an ant is lower than the `best_cost` found so far, it updates `best_route` and `best_cost`. At the end of each iteration, it calls the update_pheromone function to update the pheromone trail based on the routes taken by the ants and their associated costs.

The function then prints out the best route and its associated cost, updates the inventory levels based on the best route and returns a Pandas dataframe containing the updated inventory levels.

Finally, the `run_aco_independent()` function is called at the bottom of the code to run the ACO algorithm and print the results.


## 2.6. Algorithm: Running the Independent ACO Algorithm for Store Routing Optimization


### Inputs

- `num_iterations`: number of iterations to run the algorithm
- `num_ants`: number of ants used by the algorithm
- `num_stores`: number of stores in the network
- `lead_time`: 1D array of length `num_stores` with shipping time from each store to other stores
- `demand`: 1D array of length `num_stores` with demand at each store
- `inventory`: 1D array of length `num_stores` with initial inventory levels at each store
- `shipping_cost`: 2D array of shape `(num_stores, num_stores)` with shipping cost from each store to other stores
- `replenishment_cost`: 1D array of length `num_stores` with replenishment cost at each store
- `alpha`: parameter controlling importance of pheromone trails in ant decision making
- `beta`: parameter controlling importance of distance/cost in ant decision making
- `rho`: evaporation rate of pheromone trails
- `q`: constant used in update of pheromone trails

### Outputs

- `best_route`: 1D array of length `num_stores` with the order in which stores should be visited to minimize total cost
- `best_cost`: the total cost (including shipping and replenishment costs) associated with the best route found by the algorithm
- `inventory`: 1D array of length `num_stores` with final inventory levels at each store after running the algorithm

### Steps

1. Set `best_route` to None and `best_cost` to infinity.

2. For `iteration` in `range(num_iterations)`:

    a. Create a 2D array `ant_routes` of shape `(num_ants, num_stores)` filled with zeroes.

    b. Create a 1D array `ant_costs` of length `num_ants` filled with zeroes.

    c. For `i` in `range(num_ants)`:

    i. Choose a random starting store `current_store` from 0 to `num_stores-1`.

    ii. Set `visited_stores` to `[current_store]`.

    iii. For `j` in `range(num_stores-1)`:

    1. Calculate the probability of choosing each unvisited store using the `prob()` function.

    2. Use numpy's `random.choice()` function to choose the next store based on the calculated probabilities.

    3. Add `next_store` to `ant_routes[i]` at index `j+1` (since index 0 is reserved for the starting store).

    4. Calculate the shipping time from `current_store` to `next_store` and use it to calculate `order_qty` as `demand[next_store] - inventory[next_store]`.

    5. If `order_qty` is negative, set it to zero.

    6. Calculate the shipping cost to `next_store` and replenishment cost for `next_store` and add them to `ant_costs[i]`.

    7. Append `next_store` to `visited_stores` and set `current_store` to `next_store`.

    iv. If `ant_costs[i]` is less than `best_cost`, update `best_route` to `ant_routes[i]` and `best_cost` to `ant_costs[i]`.

    d. Update pheromone trails using the `update_pheromone()` function.

3. Print `best_route` and `best_cost`.

4. For `i` in `range(num_stores)`, update `inventory level` at `i`th store by adding (demand[i] - inventory[i]) multiplied by 1 if i is in best_route else 0.`

5. Return updated inventory levels.

In [62]:
# Define main function to run the ACO algorithm
def run_aco_independent():
    # Initialize best route and cost
    best_route = None
    best_cost = np.inf
    # Run ACO algorithm
    for iteration in range(num_iterations):
        ant_routes = np.zeros((num_ants, num_stores), dtype=int)
        ant_costs = np.zeros(num_ants)
        for i in range(num_ants):
            current_store = np.random.randint(num_stores)
            visited_stores = [current_store]
            for j in range(num_stores-1):
                probabilities = prob(current_store, visited_stores, pheromone, alpha, beta)
                next_store = np.random.choice(np.arange(num_stores)[np.isin(np.arange(num_stores), visited_stores)==False], p=probabilities)
                ant_routes[i,j+1] = next_store
                shipping_time = lead_time[next_store]
                order_qty = demand[next_store] - inventory[next_store]
                if order_qty < 0:
                    order_qty = 0
                shipping_cost_to_next_store = shipping_cost[current_store, next_store] * order_qty
                replenishment_cost_for_next_store = replenishment_cost[next_store] * order_qty
                ant_costs[i] += shipping_cost_to_next_store + replenishment_cost_for_next_store
                visited_stores.append(next_store)
                current_store = next_store
            if ant_costs[i] < best_cost:
                best_route = ant_routes[i]
                best_cost = ant_costs[i]
        update_pheromone(pheromone, ant_routes, ant_costs, rho, q)

    # Str representation of the best route        
    best_route_str = ""
    for store in best_route:
        best_route_str += f"Store N°{store} -> "
    best_route_str = best_route_str[:-3]

    # Display the best route and cost
    print(f"Best Route: {best_route_str}\n")
    print(f"Best cost: {best_cost:.2f}")

    # Update inventory levels based on best route
    inventory1 = inventory.copy()
    for i in range(num_stores):
        inventory1[i] += (demand[i] - inventory1[i]) * int(i in best_route)
    
    # Return updated inventory levels
    Inventory_df1 = pd.DataFrame({'Store': np.arange(num_stores), 'Demand-Optimal Inventory': inventory1})
    display(Inventory_df1)
    
    return best_cost

In [64]:
# Now it's time we apply our algorithm on an example !
print("**************** Using Independant Ants ****************")
start1 = time.time()
best_cost1 = run_aco_independent()
end1 = time.time()
print(f"The processing time is: {end1-start1:.2f}s")

**************** Using Independant Ants ****************
Best Route: Store N°0 -> Store N°43 -> Store N°17 -> Store N°8 -> Store N°9 -> Store N°19 -> Store N°11 -> Store N°34 -> Store N°22 -> Store N°21 -> Store N°41 -> Store N°35 -> Store N°2 -> Store N°31 -> Store N°24 -> Store N°39 -> Store N°16 -> Store N°12 -> Store N°14 -> Store N°4 -> Store N°42 -> Store N°10 -> Store N°32 -> Store N°6 -> Store N°46 -> Store N°27 -> Store N°7 -> Store N°23 -> Store N°49 -> Store N°44 -> Store N°33 -> Store N°47 -> Store N°45 -> Store N°36 -> Store N°15 -> Store N°0 -> Store N°38 -> Store N°20 -> Store N°28 -> Store N°25 -> Store N°5 -> Store N°37 -> Store N°40 -> Store N°30 -> Store N°18 -> Store N°3 -> Store N°48 -> Store N°1 -> Store N°26 -> Store N°13 

Best cost: 43.85


,Store,Inventory
0,0,22
1,1,70
2,2,28
3,3,13
4,4,27
5,5,46
6,6,81
7,7,50
8,8,14
9,9,35


The processing time is: 10.62s


## 2.7. Running the parallel ACO algorithm


### 2.7.1. The process for each ant



The algorithm behind the `run_ant(current_store)` function:

- Takes the `current store` as input and generates a route for a single ant using the ACO algorithm
- Initializes `visited_stores` with current_store and `ant_route` with a zero-filled numpy array of size `num_stores`
- Iterates over the remaining stores and calculates the `probabilities` of choosing the next store based on pheromone and heuristic information
- Chooses the `next store` using roulette wheel selection
- Calculates `shipping time`, `order quantity`, `shipping cost`, and `replenishment cost` for the chosen next store and updates the `ant_cost`
- Updates the `inventory level` for the chosen next store and sets `current_store` to the chosen next store
- Revisits the `last store` and calculates `shipping time`, `shipping cost`, and `replenishment cost` for the last store, adding them to `ant_cost`
- Returns `ant_route` and `ant_cost`

In [65]:
# Define function for each ant to run
def run_ant(current_store):
    visited_stores=[current_store]
    ant_route=np.zeros(num_stores,dtype=int)
    ant_route[0]=current_store
    ant_cost=0

    for j in range(num_stores-1):
        probabilities=prob(current_store,visited_stores,pheromone,alpha,beta)
        next_store=np.random.choice(np.arange(num_stores)[np.isin(np.arange(num_stores),visited_stores)==False],p=probabilities)
        ant_route[j+1]=next_store

        shipping_time=lead_time[next_store]
        order_qty=demand[next_store]-inventory[next_store]
        if order_qty<0:
            order_qty=0
        shipping_cost_to_next_store=shipping_cost[current_store,next_store]*order_qty
        replenishment_cost_for_next_store=replenishment_cost[next_store]*order_qty
        ant_cost+=shipping_cost_to_next_store+replenishment_cost_for_next_store
        inventory[next_store]+=order_qty
        visited_stores.append(next_store)
        current_store=next_store
        ant_cost+=shipping_time*shipping_cost_to_next_store

    last_store=ant_route[-1]
    shipping_time=lead_time[current_store]
    shipping_cost_to_last_store=shipping_cost[current_store,last_store]*demand[last_store]
    ant_cost+=shipping_time*shipping_cost_to_last_store
    ant_route[-1]=last_store
    ant_cost+=replenishment_cost[last_store]*demand[last_store]

    # Return ant_route and ant_cost
    return ant_route.tolist(), ant_cost

### 2.7.2. The parallel processing algorithm

The `run_aco_parallel()` function:

- Runs the ACO algorithm in parallel by generating routes for all ants in parallel using `Parallel` class from `joblib` library
- Initializes `best_cost` to infinity and `best_route` to an empty list
- Runs the ACO algorithm for a specified number of iterations, `updating` the `pheromone trail` and finding the `best route` and `cost` at each iteration
- Computes the updated pheromone trail using the update_pheromone function
- Updates `best_route` and `best_cost` if the current ant's cost is less than `best_cost`
- Returns `best_cost` and displays `best_cost` and `best_route`
- Updates `inventory levels` based on the best route and returns the updated inventory levels as a pandas DataFrame

In [66]:
# Run the ACO algorithm
def run_aco_parallel() :
    best_cost = np.inf
    best_route = []
    for it in range(num_iterations):
        # Generate routes for all ants in parallel
        with parallel_backend('loky', n_jobs=5):
            routes = Parallel()(delayed(run_ant)(i) for i in range(num_ants))
        ant_routes = np.array([route[0] for route in routes],dtype=int)
        ant_costs = np.array([route[1] for route in routes])
        # Update pheromone trail
        update_pheromone(pheromone, ant_routes, ant_costs, rho, q)
        # Find the best route
        min_cost_idx = np.argmin(ant_costs)
        if ant_costs[min_cost_idx] < best_cost:
            best_cost = ant_costs[min_cost_idx]
            best_route = ant_routes[min_cost_idx]

    # Str representation of the best route        
    best_route_str = ""
    for store in best_route:
        best_route_str += f"Store N°{store} -> "
    best_route_str = best_route_str[:-3]

    # Display the best route and cost
    print(f"Best Route: {best_route_str}\n")
    print(f"Best Cost: {best_cost:.2f}")
    
    # Update inventory levels based on best route
    inventory2 = inventory.copy()
    for i in range(num_stores):
        inventory2[i] += (demand[i] - inventory2[i]) * int(i in best_route)
    
    # Return updated inventory levels
    Inventory_df2 = pd.DataFrame({'Store': np.arange(num_stores), 'Demand-Optimal Inventory': inventory2})
    display(Inventory_df2)
    
    return best_cost


In [67]:
# Now it's time we apply our algorithm on an example !
print("**************** Using Parallel Ants ****************")
start2 = time.time()
best_cost2 = run_aco_parallel()
end2 = time.time()
print(f"The processing time is: {end2-start2:.2f}s")

**************** Using Parallel Ants ****************
Best Route: Store N°19 -> Store N°18 -> Store N°21 -> Store N°44 -> Store N°11 -> Store N°12 -> Store N°17 -> Store N°35 -> Store N°16 -> Store N°9 -> Store N°6 -> Store N°2 -> Store N°31 -> Store N°30 -> Store N°45 -> Store N°49 -> Store N°23 -> Store N°15 -> Store N°26 -> Store N°36 -> Store N°39 -> Store N°14 -> Store N°29 -> Store N°25 -> Store N°3 -> Store N°1 -> Store N°8 -> Store N°13 -> Store N°28 -> Store N°41 -> Store N°0 -> Store N°47 -> Store N°40 -> Store N°27 -> Store N°42 -> Store N°37 -> Store N°33 -> Store N°22 -> Store N°20 -> Store N°5 -> Store N°48 -> Store N°24 -> Store N°10 -> Store N°7 -> Store N°43 -> Store N°46 -> Store N°4 -> Store N°32 -> Store N°38 -> Store N°34 

Best Cost: 4.05


,Store,Inventory
0,0,22
1,1,70
2,2,28
3,3,13
4,4,27
5,5,46
6,6,81
7,7,50
8,8,14
9,9,35


The processing time is: 4.69s


## 2.8. Comparing the findings



The results of the experiment show that using parallel ants in the ant colony optimization (ACO) algorithm significantly improved the performance of the algorithm. Specifically, the best route cost was cut down roughly by 80%, indicating that the ACO algorithm with parallel ants was able to find a much better solution than the ACO algorithm without parallel ants. Additionally, the processing time was reduced by roughly 60% (depending on the machinery: here it was run on an 8 core & 4.3/3.1CHz CPU), which is a significant improvement in terms of the computational efficiency of the algorithm. These results demonstrate the effectiveness of parallelization in improving the performance of optimization algorithms, and highlight the potential benefits of using parallelization in other optimization applications. 

In [68]:
print("**************** comparison ****************")
print(f"The best route cost was reduced by {(best_cost1-best_cost2)*100/best_cost1:.2f}%")
print(f"The processing time was reduced by {((end1-start1-(end2-start2)))*100/(end1-start1):.2f}%")

**************** comparison ****************
The best route cost was reduced by 90.77%
The processing time was reduced by 55.80%


# 3. Conclusion



Ant Colony Optimization is a highly helpful technique that helps accelerate supply chains, to sum up. It is a promising strategy that has the potential to revolutionize supply chain optimization and make it more effective and efficient in the future and in our case, it has demonstrated to make supply chain networks more efficient. That's why future research is necessary to focus on further improving the performance and robustness of ACO for supply chain optimization.